#### Let us start by importing our dependencies

In [1]:
import pandas as pd
import numpy as np
from config import api_key
import requests
import json
import scipy.stats as stats

In [2]:
# Create a dataframe of all award-winning beers since 2009
csv_data = "beer_winners.csv"

all = pd.read_csv(csv_data)
all.columns = ['medal', 'beer_name','brewery',
                       'city','county','state',
                       'category','year','total_category_entries']
all = all.sort_values('year', ascending=False)
all.reset_index(inplace=True)
all_winner_data = all.drop(columns=['index'],axis=1)
print("This is a dataframe with every award-winning beer since 2009")
all_winner_data.head()

This is a dataframe with every award-winning beer since 2009


,medal,beer_name,brewery,city,county,state,category,year,total_category_entries
0,Bronze,Eclipse,FiftyFifty Brewing Co.,Truckee,Nevada,CA,Wood- and Barrel-Aged Strong Stout,2018,177
1,Silver,Märzen,Sudwerk Brewing Co.,Davis,Yolo,CA,American-Style Amber Lager,2018,91
2,Gold,Oktoberfest,SKA Brewing,Durango,La Plata,CO,Vienna-Style Lager,2018,92
3,Gold,Barrel Aged Japance Off,Denver Beer Co. - Olde Town Arvada,Arvada,Jefferson,CO,Wood- and Barrel-Aged Beer,2018,77
4,Gold,3 Barrel Circus,River Dog Brewing Co.,Bluffton,Yell,SC,Wood- and Barrel-Aged Sour Beer,2018,76


### How many beers brewed in California have won a medal?

In [3]:
ca = all_winner_data[all_winner_data['state'] == 'CA']
ca.reset_index(inplace=True)
ca_number = ca['index'].count()
print(f'There have been {ca_number} award-winning* beers in California in the last decade.')
cal_data = ca.drop(columns=['index'],axis=1)
cal_data = cal_data.sort_values('year', ascending=False)
cal_data.reset_index(inplace=True)
cal_data = cal_data.drop(columns=['index'],axis=1)
cal_data.head()

There have been 553 award-winning* beers in California in the last decade.


,medal,beer_name,brewery,city,county,state,category,year,total_category_entries
0,Bronze,Eclipse,FiftyFifty Brewing Co.,Truckee,Nevada,CA,Wood- and Barrel-Aged Strong Stout,2018,177
1,Bronze,Nighthawk,Enegren Brewing Co.,Moorpark,Ventura,CA,Dark Lager,2018,62
2,Silver,Sour Cherry Sour,Hermitage Brewing Co.,San Jose,Graham,CA,Fruited Wood- and Barrel-Aged Sour Beer,2018,102
3,Bronze,Kumquat Saison,Smog City Brewing Wood Cellar,Torrance,Los Angeles,CA,Belgian-Style Fruit Beer,2018,65
4,Silver,The Shroud,Bravery Brewing,Lancaster,Los Angeles,CA,Imperial Stout,2018,86


### How many beers brewed in San Diego have won a medal?

In [4]:
temp = cal_data[cal_data['county'] == 'San Diego']
temp.reset_index(inplace=True)
sd_number = temp['index'].count()
print(f'There have been {sd_number} award-winning* beers in San Diego in the last decade.')
sd_data = temp.drop(columns=['index'], axis=1)
sd_data = sd_data.sort_values('year', ascending=False)
sd_data.reset_index(inplace=True)
sd_data = sd_data.drop(columns=['index'],axis=1)
sd_data.head(300)

There have been 148 award-winning* beers in San Diego in the last decade.


,medal,beer_name,brewery,city,county,state,category,year,total_category_entries
0,Silver,Oats,Pizza Port Solana Beach,Solana Beach,San Diego,CA,Oatmeal Stout,2018,59
1,Gold,Natural Bridge: Vienna Lager,Eppig Brewing,San Diego,San Diego,CA,Session Beer,2018,50
2,Gold,Liquid AC,Karl Strauss Brewing Co. - La Jolla,La Jolla,San Diego,CA,English-Style Summer Ale,2018,61
3,Bronze,Bird Park,North Park Beer Co.,San Diego,San Diego,CA,Bohemian-Style Pilsener,2018,84
4,Bronze,Chocolate Macadamia Nut Stout,Kilowatt Brewing,San Diego,San Diego,CA,Chocolate Beer,2018,59
5,Gold,Rye Dawn,Breakwater Brewing Co.,Oceanside,San Diego,CA,Rye Beer,2018,57
6,Silver,Groundswell Piloncillo Brown Ale,Groundswell Brewing Co.,Santee,San Diego,CA,American-Style Brown Ale,2018,76
7,Bronze,Know Ego,Groundswell Brewing Co.,Santee,San Diego,CA,Classic Irish-Style Dry Stout,2018,54
8,Bronze,Rail Gun Wee Heavy,BNS Brewing & Distilling Co.,Santee,San Diego,CA,Old Ale or Strong Ale,2018,37
9,Gold,The Coachman,Societe Brewing Co.,San Diego,San Diego,CA,Session India Pale Ale,2018,92


### In terms of medals won, what are the top 5 breweries in San Diego?

In [5]:
medals_sd = sd_data.groupby(['brewery'])['medal'].count()
top_brew_sd = medals_sd.sort_values(ascending=False)
top_5_sd = top_brew_sd.head()
top_5_sd

brewery
Pizza Port Carlsbad        24
AleSmith Brewing Co.       10
The Lost Abbey              9
Pizza Port Solana Beach     8
Pizza Port Ocean Beach      7
Name: medal, dtype: int64

### In terms of medals won, what are the top 5 breweries in California?

In [6]:
medals_cal = cal_data.groupby(['brewery'])['medal'].count()
top_brew_cal = medals_cal.sort_values(ascending=False)
top_5_cal = top_brew_cal.head()
top_5_cal

brewery
Firestone Walker Brewing Co.    28
Pizza Port Carlsbad             24
Pabst Brewing Co.               12
Pizza Port Ocean Beach          12
Beachwood BBQ & Brewing         11
Name: medal, dtype: int64




### Why is there a discrepancy between the San Diego list and the California list?
#### Let's find out!




In [7]:
pizza_port = all_winner_data[all_winner_data['brewery'] == 'Pizza Port Ocean Beach']
pizza_port = pizza_port.sort_values('year', ascending=False)
pizza_port.reset_index(inplace=True)
pizza_port = pizza_port.drop(columns=['index'],axis=1)
pizza_port

,medal,beer_name,brewery,city,county,state,category,year,total_category_entries
0,Gold,Guillaume,Pizza Port Ocean Beach,San Diego,San Diego,CA,Session Beer,2017,40
1,Bronze,Bacon and Eggs,Pizza Port Ocean Beach,San Diego,San Diego,CA,Coffee Beer,2015,149
2,Bronze,Guillaume,Pizza Port Ocean Beach,San Diego,San Diego,CA,Session Beer,2015,44
3,Silver,Guillaume,Pizza Port Ocean Beach,Ocean Beach,Suffolk,CA,Session Beer,2014,94
4,Bronze,La Flama Dorada,Pizza Port Ocean Beach,Ocean Beach,Suffolk,CA,Belgian-Style Tripel,2014,58
5,Gold,Beer Hunter,Pizza Port Ocean Beach,Ocean Beach,Suffolk,CA,Session Beer,2013,56
6,Silver,Rhino Chaser,Pizza Port Ocean Beach,Ocean Beach,Suffolk,CA,Imperial Red Ale,2013,56
7,Bronze,Kung Fu Elvis,Pizza Port Ocean Beach,Ocean Beach,Suffolk,CA,American-Style Strong Pale Ale,2013,120
8,Gold,Skidmark Brown Ale,Pizza Port Ocean Beach,San Diego,San Diego,CA,English-Style Mild Ale,2011,25
9,Bronze,Z-Man Stout,Pizza Port Ocean Beach,San Diego,San Diego,CA,Foreign-Style Stout,2011,23


### Now for the big question:
### What are the top 5 breweries in the United States?

In [8]:
medals_us = all_winner_data.groupby(['brewery'])['medal'].count()
top_brew_us = medals_us.sort_values(ascending=False)
top_5_us = top_brew_us.head()
top_5_us

brewery
Firestone Walker Brewing Co.    28
Pizza Port Carlsbad             24
Sun King Brewing Co.            19
Pabst Brewing Co.               19
Chuckanut Brewery               18
Name: medal, dtype: int64

In [9]:
# ^^^
# Maybe we might want to speak on the fact that
# Firestone is in SLO and Pizza Port Carlsbad is in SD County?

In [10]:
# Create a dataframe?
# Possible visualization ideas?
# Possible state-by-state breakdown of medals? Using a different groupby?
# Even see the mean of all US states medal counts and compare it to CA? <-- I like this idea the best so far, myself

sd_medals = sd_data.groupby(['medal'])['year'].count()
ca_medals = cal_data.groupby(['medal'])['year'].count()
all_medals = all_winner_data.groupby(['medal'])['year'].count()

medals_df = pd.DataFrame({"SD medals won": sd_medals,
                         "CA medals won": ca_medals,
                         "All US won": all_medals})

medals_df['SD expected'] = [49, 49, 49]
medals_df['CA expected'] = [184, 184, 184]
medals_df['US expected'] = [882, 882, 882]

In [11]:
sd_df = medals_df[['SD medals won', 'SD expected']]
sd_df

,SD medals won,SD expected
medal,,
Bronze,49,49
Gold,58,49
Silver,41,49


In [12]:
crit_sd = stats.chi2.ppf(q = 0.95, df = 2)
print(f'The critical value for medal frequency is {crit_sd}')
stats.chisquare(sd_df['SD medals won'], sd_df['SD expected'])

The critical value for medal frequency is 5.991464547107979


Power_divergenceResult(statistic=2.9591836734693877, pvalue=0.2277306206904969)

### Conclusion
* Since the chi-square value of 2.96 at a confidence level of 95% does not exceed the critical value of 5.99, we conclude that the differences seen in the number of wins per medal are not statistically significant.
* If you were handed a randomly-chosen award-winning beer in San Diego, CA, it is just as likely to be a gold-medal beer as a bronze-medal beer.


In [13]:
ca_df = medals_df[['CA medals won', 'CA expected']]
ca_df

,CA medals won,CA expected
medal,,
Bronze,186,184
Gold,185,184
Silver,182,184


In [14]:
crit_ca = stats.chi2.ppf(q = 0.95, df = 2)
print(f'The critical value for medal frequency is {crit_ca}')
stats.chisquare(ca_df['CA medals won'], ca_df['CA expected'])

The critical value for medal frequency is 5.991464547107979


Power_divergenceResult(statistic=0.048913043478260865, pvalue=0.9758401158271994)

### Conclusion
* Since the chi-square value of 0.05 at a confidence level of 95% does not exceed the critical value of 5.99, we conclude that the differences seen in the number of wins per medal are not statistically significant.
* With a p-value of 0.976, the distribution of medals won throughout all of California is roughly [uniform](http://mathworld.wolfram.com/UniformDistribution.html).
* If you were handed a randomly-chosen award-winning beer from California, there is a 1/3 chance of it winning a gold medal, a 1/3 chance of it winning a silver medal, and a 1/3 chance of it winning a bronze medal.


In [15]:
means_us = all_winner_data.groupby(['category'])['total_category_entries'].mean()
means_us.head(10)

category
Aged Beer                                    33.2
American Style Amber Lager                   45.0
American Style Amber/Red Ale                 78.0
American Style Brown Ale                     43.0
American Style Light Lager                   25.0
American Style Pale Ale                     108.0
American Style Specialty Lager               23.0
American-Belgo-Style Ale                     60.7
American-Style Amber Lager                   58.0
American-Style Amber Lager or Dark Lager    108.0
Name: total_category_entries, dtype: float64

In [16]:
means_cal = cal_data.groupby(['category'])['total_category_entries'].mean()
means_cal.head(10)

category
Aged Beer                                    34.333333
American Style Amber/Red Ale                 78.000000
American Style Brown Ale                     43.000000
American-Belgo-Style Ale                     63.375000
American-Style Amber Lager                   69.500000
American-Style Amber Lager or Dark Lager    108.000000
American-Style Amber/Red Ale                 96.363636
American-Style Black Ale                     63.625000
American-Style Brett Beer                    38.000000
American-Style Brown Ale                     75.142857
Name: total_category_entries, dtype: float64

In [17]:
means_sd = sd_data.groupby(['category'])['total_category_entries'].mean()
means_sd.head(10)

category
American Style Amber/Red Ale       78.000000
American Style Brown Ale           43.000000
American-Belgo-Style Ale           70.333333
American-Style Amber/Red Ale       99.333333
American-Style Black Ale           73.000000
American-Style Brown Ale           69.666667
American-Style India Black Ale     53.000000
American-Style India Pale Ale     235.000000
American-Style Pale Ale           157.500000
American-Style Sour Ale            53.400000
Name: total_category_entries, dtype: float64

In [18]:
# Possibly a line graph? Or scatter-plot?

total_entries_us = all_winner_data.groupby(['year'])['total_category_entries'].count()
print('total US entries')
print(total_entries_us)
total_entries_cal = cal_data.groupby(['year'])['total_category_entries'].count()
print("")
print('total CA entries')
print(total_entries_cal)
total_entries_sd = sd_data.groupby(['year'])['total_category_entries'].count()
print("")
print('total SD entries')
print(total_entries_sd)

total US entries
year
2009    234
2010    236
2011    248
2012    251
2013    249
2014    268
2015    275
2016    286
2017    293
2018    306
Name: total_category_entries, dtype: int64

total CA entries
year
2009    39
2010    52
2011    51
2012    49
2013    52
2014    46
2015    67
2016    68
2017    57
2018    72
Name: total_category_entries, dtype: int64

total SD entries
year
2009    14
2010    15
2011    18
2012    14
2013     8
2014    12
2015    19
2016    18
2017    14
2018    16
Name: total_category_entries, dtype: int64


In [19]:
# List of all award-winning beers in 2018
# Any ides for visualization and/or story-telling?

bweh = sd_data.loc[sd_data['year'] == 2018]
sd_2018 = bweh[['medal', 'beer_name', 'brewery', 'category', 'total_category_entries']]
sd_2018

,medal,beer_name,brewery,category,total_category_entries
0,Silver,Oats,Pizza Port Solana Beach,Oatmeal Stout,59
1,Gold,Natural Bridge: Vienna Lager,Eppig Brewing,Session Beer,50
2,Gold,Liquid AC,Karl Strauss Brewing Co. - La Jolla,English-Style Summer Ale,61
3,Bronze,Bird Park,North Park Beer Co.,Bohemian-Style Pilsener,84
4,Bronze,Chocolate Macadamia Nut Stout,Kilowatt Brewing,Chocolate Beer,59
5,Gold,Rye Dawn,Breakwater Brewing Co.,Rye Beer,57
6,Silver,Groundswell Piloncillo Brown Ale,Groundswell Brewing Co.,American-Style Brown Ale,76
7,Bronze,Know Ego,Groundswell Brewing Co.,Classic Irish-Style Dry Stout,54
8,Bronze,Rail Gun Wee Heavy,BNS Brewing & Distilling Co.,Old Ale or Strong Ale,37
9,Gold,The Coachman,Societe Brewing Co.,Session India Pale Ale,92


In [20]:
# POSSIBLE DATA VISUALIZATION FOR THIS LIST?
# ***How many award-winning beers does each brewery have?***

# Maybe a pie chart showing the medal breakdown of SD breweries?
# Or a pie chart showing the top 5-7 breweries, with all the others listed as "Other"

# Maybe combine similar names (i.e. "Ballast Point Brewing Co" and "Ballast Point Brewing Co.")
top_brew_sd.head()

brewery
Pizza Port Carlsbad        24
AleSmith Brewing Co.       10
The Lost Abbey              9
Pizza Port Solana Beach     8
Pizza Port Ocean Beach      7
Name: medal, dtype: int64

In [21]:
sd_data.head()

,medal,beer_name,brewery,city,county,state,category,year,total_category_entries
0,Silver,Oats,Pizza Port Solana Beach,Solana Beach,San Diego,CA,Oatmeal Stout,2018,59
1,Gold,Natural Bridge: Vienna Lager,Eppig Brewing,San Diego,San Diego,CA,Session Beer,2018,50
2,Gold,Liquid AC,Karl Strauss Brewing Co. - La Jolla,La Jolla,San Diego,CA,English-Style Summer Ale,2018,61
3,Bronze,Bird Park,North Park Beer Co.,San Diego,San Diego,CA,Bohemian-Style Pilsener,2018,84
4,Bronze,Chocolate Macadamia Nut Stout,Kilowatt Brewing,San Diego,San Diego,CA,Chocolate Beer,2018,59


In [22]:
group = sd_data.groupby('category')['total_category_entries']
aa = group.count()
aa[aa > 2]

#bb = sd_data.groupby('category')['total_category_entries'].mean()
#bb
# cat = group.size()
# cat_df = pd.DataFrame(cat)
# aa = cat.loc[cat_df[0] > 2]
# aa

category
American-Belgo-Style Ale              3
American-Style Amber/Red Ale          3
American-Style Brown Ale              3
American-Style Sour Ale               5
American-Style Stout                  3
American-Style Strong Pale Ale        3
Barley Wine-Style Ale                 4
Belgian-Style Abbey Ale               3
Belgian-Style Strong Specialty Ale    3
Classic Irish-Style Dry Stout         4
Coffee Beer                           4
English-Style Mild Ale                3
English-Style Summer Ale              3
Imperial India Pale Ale               3
Imperial Red Ale                      6
Irish-Style Red Ale                   3
Oatmeal Stout                         4
Old Ale or Strong Ale                 3
Other Belgian-Style Ale               3
Scotch Ale                            4
Session Beer                          9
Session India Pale Ale                3
Name: total_category_entries, dtype: int64